Equação (https://pubs.acs.org/doi/full/10.1021/ja401184g):

$P_d = \sum_i \sum_j P_i P_j \delta(d - d_{ij})$



In [ ]:
import numpy as np
import pandas as pd
from rdkit import Chem

In [ ]:
elem_data = pd.read_csv('Atomic properties DB.csv')

In [ ]:
#Converting the df into a dictionary to be used in the Kulik's descriptor

'''
    Converting DataFrame to Dicionary using a COLUMN as KEY, in the format:

    prop: {<ELEMENT_1>: (prop1, prop2, prop3,...),
            <ELEMENT_2>: (prop1, prop2, prop3,...),...
            }

    Lengh of list of properties: 10

    Index of each property:

        0: index
        1: Z
        2: Name
        3: Average atomic mass
        4: Electronegativity (Pauling)
        5: First Ionization Energy (eV)
        6: Atomic Radii (pm)
        7: Van der Waals Radii (pm)
        8: Covalent Radii (pm)
        9: Valence electrons
'''

properties_dict = elem_data.set_index('Symbol').T.to_dict('list')
print(properties_dict)

{'H': [1, 'Hydrogen', 1.007, 2.2, 1359844.0, 25.0, 120.0, 38, 1.0], 'He': [2, 'Helium', 4.002602, nan, 2458741.0, 31.0, 140.0, 32, 2.0], 'Li': [3, 'Lithium', 6.941, 0.98, 539172.0, 145.0, 182.0, 134, 1.0], 'Be': [4, 'Beryllium', 9.012182, 1.57, 9.3227, 105.0, nan, 90, 2.0], 'B': [5, 'Boron', 10.811, 2.04, 829803.0, 85.0, nan, 82, 3.0], 'C': [6, 'Carbon', 12.0107, 2.55, 11.2603, 70.0, 170.0, 77, 4.0], 'N': [7, 'Nitrogen', 14.0067, 3.04, 1453414.0, 65.0, 155.0, 75, 5.0], 'O': [8, 'Oxygen', 15.9994, 3.44, 1361806.0, 60.0, 152.0, 73, 6.0], 'F': [9, 'Fluorine', 18.9984032, 3.98, 1742282.0, 50.0, 147.0, 71, 7.0], 'Ne': [10, 'Neon', 20.1797, nan, 215646.0, 38.0, 154.0, 69, 8.0], 'Na': [11, 'Sodium', 22.98976928, 0.93, 513908.0, 180.0, 227.0, 154, 1.0], 'Mg': [12, 'Magnesium', 24.305, 1.31, 764624.0, 150.0, 173.0, 130, 2.0], 'Al': [13, 'Aluminium', 26.9815386, 1.61, 598577.0, 125.0, nan, 118, 3.0], 'Si': [14, 'Silicon', 28.0855, 1.9, 815169.0, 110.0, 210.0, 111, 4.0], 'P': [15, 'Phosphorus', 3

In [ ]:
def create_property_descriptors(smiles, depth, prop, prop_index=0):
    '''
    Função:
        create_property_descriptors(smiles, depth, prop, prop_index=0)

    Descrição:
        Permite criar descritores baseados em propriedades atômicas e conectividade básica entre os átomos

    Argumentos:
        smiles:     string SMILES da molécula que se deseja gerar os descritores. Exemplo: 'C(C)CO'.
        depth:      profundidade, parâmetro que permite definir o nivel máximo de conectividade que será
                    usado na geração dos descritores. Exemplo, se depth=3, três descritores serão gerados,
                    com conectividade iguais a um, dois e três.
        prop:       dicionário formado por pares 'átomo': propriedade ou 'átomo':[propriedades] ou
                    'átomo':(propriedades).
        prop_index: índice da propriedade de interesse nos valores de 'prop'. Se estes valores não forem
                    listas ou tuplas, este parâmetro é ignorado. Se forem, este parâmetro deve ser
                    configurado com um número maior ou igual a zero e menor do que o número de itens nas
                    listas ou tuplas. O valor default deste parâmetro é zero, ou seja, a função pegará a
                    primeira propriedade da lista.
    '''
    # Criar a estrutura a partir do SMILES da molécula, adicionando os H faltantes
    mol = Chem.MolFromSmiles(smiles)
    mol = Chem.AddHs(mol)

    # Preparar listas de resultados (separação por depth=0, 1, 2, 3...)
    result_list = [0] * (depth+1) # Se depth = 2, precisamos de 3 itens (para d=[0,1,2])

    # Obter a matriz de distâncias entre cada um dos átomos e a lista de átomos
    atom_list = [ atom.GetSymbol() for atom in mol.GetAtoms() ]
    n_atoms = len(atom_list)
    dist_matrix = np.tril(Chem.rdmolops.GetDistanceMatrix(mol, force=True))

    # Iterar toda a matriz, extraindo os dados de interesse e separando por valor de depth
    for i in range(0, n_atoms):
        for j in range(0, n_atoms):
            d = int(dist_matrix[i][j])

            if d <= depth and d == 0 and i == j:
                a_i = atom_list[i]
                p_i = 0

                if prop[a_i] is list or tuple:
                    p_i = prop[a_i][prop_index]   # caso haja + de 1 propriedade p/ calcular

                else:
                    p_i = prop[a_i]            # caso haja somente 1 propriedade p/ calcular

                result_list[d] += round(p_i * p_i, 3)

            if d <= depth and d > 0:
                a_i = atom_list[i]
                a_j = atom_list[j]
                p_i, p_j = 0, 0

                if prop[a_i] is list or tuple:
                    p_i = prop[a_i][prop_index]   # caso haja + de 1 propriedade p/ calcular
                else:
                    p_i = prop[a_i]            # caso haja somente 1 propriedade p/ calcular
                if prop[a_j] is list or tuple:
                    p_j = prop[a_j][prop_index]
                else:
                    p_j = prop[a_j]
                result_list[d] += round(p_i * p_j, 3)    # posição [d] na lista de resultados é incrementada

    # Finalizando
    return result_list

In [ ]:
create_property_descriptors('C', 3, properties_dict, 3)

[25.862, 22.44, 29.04, 0]